<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Deep Autoencoder Deep Dive

In [32]:
import sys
sys.path.append("../../")
import os
import numpy as np
import pandas as pd
import torch
import copy

from reco_utils.common.gpu_utils import get_number_gpus, get_cuda_version
from reco_utils.common.general_utils import get_number_processors
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import rmse, mae, rsquared, exp_var
from reco_utils.recommender.deep_autoencoder.autoencoder import AutoEncoder
from reco_utils.recommender.deep_autoencoder.data import UserItemRecDataProvider
from reco_utils.recommender.deep_autoencoder.utils import add_gpu, init_optimizer, MSEloss

import logging
log = logging.getLogger(__name__)

print("OS:", sys.platform)
print("Python: ", sys.version)
print("PyTorch:", torch.__version__)
print("Number of CPU processors:", get_number_processors())
print("Number of GPUs:", get_number_gpus())
print(get_cuda_version())

%load_ext autoreload
%autoreload 2

OS: linux
Python:  3.6.7 | packaged by conda-forge | (default, Nov 21 2018, 03:09:43) 
[GCC 7.3.0]
PyTorch: 1.0.0
Number of CPU processors: 6
Number of GPUs: 1
CUDA Version 9.2.148
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
train_path = "ratings_train.csv"
valid_path = "ratings_valid.csv"
test_path = "ratings_test.csv"


In [7]:
data_params = {'batch_size': 128,
                'major': 'users',  # major position is the first column id of input data
                'itemIdInd': 1,  # the second index is the items
                'userIdInd': 0,  # the first index is the users/customers
                'delimiter': ',',
                'header': True,
                "src_file": train_path
                }
torch.manual_seed(42)

In [11]:
data_layer = UserItemRecDataProvider(params=data_params)
#dir(data_layer)

In [14]:
eval_params = copy.deepcopy(data_params)
eval_params['src_file'] = valid_path
validation_layer = UserItemRecDataProvider(
    params=eval_params,
    user_id_map=data_layer.user_id_map,
    item_id_map=data_layer.item_id_map)
validation_layer.src_data = data_layer.data


In [19]:
hidden_layers = [1024, 512, 512, 128]
model = AutoEncoder(
    layer_sizes=[data_layer.vector_dim] + hidden_layers,
    nl_type="selu",
    is_constrained=False,
    dp_drop_prob=0.8)

../../reco_utils/recommender/deep_autoencoder/autoencoder.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)
../../reco_utils/recommender/deep_autoencoder/autoencoder.py:61: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  weight_init.xavier_uniform(w)


In [23]:
model = add_gpu(model, "0")

In [25]:
optimizer, scheduler = init_optimizer(model,
                       optimization_method="momentum",
                       lr=0.005,
                       wd=0.00001)

In [30]:
from torch.autograd import Variable


cuda_availability = True
def train_loop(rencoder, optimizer, scheduler=None):    
    """
    Internal train loop
    """
    t_loss = 0.0
    t_loss_denom = 0.0
    global_step = 0
    best_loss = sys.maxsize
    best_epoch = 0
    epoch = 0
    losing_patience = 0

    # Params
    noise_prob = 0.0
    num_epochs = 5
    aug_step = 1


    if noise_prob > 0.0:
        dp = nn.Dropout(p=noise_prob)

    # Train until finish epochs or early stoping fires
    while epoch < num_epochs and losing_patience < 10:
        log.debug('Doing epoch {} of {}'.format(epoch, num_epochs))
        rencoder.train()
        total_epoch_loss = 0.0
        denom = 0.0
        if scheduler:
            scheduler.step()
        for i, mb in enumerate(data_layer.iterate_one_epoch()):
            inputs = Variable(mb.cuda().to_dense()) if cuda_availability else Variable(mb.to_dense())
            optimizer.zero_grad()
            loss, outputs = _backprop(rencoder, inputs, optimizer)
            global_step += 1
            t_loss += loss.data[0]
            t_loss_denom += 1
            total_epoch_loss += loss.data[0]
            denom += 1

            if aug_step > 0:
                # Magic data augmentation trick happen here
                for t in range(aug_step):
                    inputs = Variable(outputs.data)
                    if noise_prob > 0.0:
                        inputs = dp(inputs)
                    optimizer.zero_grad()
                    loss, outputs = _backprop(
                        rencoder, inputs, optimizer)

        # Track model with lowest loss
        epoch_loss = sqrt(total_epoch_loss/denom)
        log.debug("Epoch {} - Training loss: {}".format(epoch, epoch_loss))
        if True:# self.params['use_validation']:
            epoch_loss = _evaluate_on_validation_set(rencoder)
            log.debug("Epoch {} - Validation loss: {}".format(epoch,
                                                              epoch_loss))
        if epoch_loss < best_loss:
            losing_patience = 0
            best_loss = epoch_loss
            best_epoch = epoch
            best_model_wts = copy.deepcopy(rencoder.state_dict())
        else:
            # early stoping
            losing_patience += 1
        epoch += 1

    # Save final model
    log.debug("Best loss {} in epoch {}".format(best_loss, best_epoch))
    #self._save_model(best_model_wts, best_epoch)
    #rencoder.load_state_dict(best_model_wts)

def _backprop(rencoder, inputs, optimizer):
    outputs = rencoder(inputs)
    loss, num_ratings = autoencoder.MSEloss(outputs, inputs)
    loss = loss / num_ratings
    loss.backward()
    optimizer.step()
    return loss, outputs

def _evaluate_on_validation_set(rencoder):
    rencoder.eval()
    denom = 0.0
    total_epoch_loss = 0.0
    for target_mb, user_profile in data_layer.iterate_one_epoch_eval():
        inputs = Variable(user_profile.cuda().to_dense()) if cuda_availability else Variable(user_profile.to_dense())
        targets = Variable(target_mb.cuda().to_dense()) if cuda_availability else Variable(target_mb.to_dense())
        outputs = rencoder(inputs)
        loss, num_ratings = autoencoder.MSEloss(outputs, targets)
        total_epoch_loss += loss.data[0]
        denom += num_ratings.data[0]
    return sqrt(total_epoch_loss / denom)

In [31]:
train_loop(model, optimizer, scheduler)

NameError: name 'autoencoder' is not defined